# Bert Study 버트가 뭐지?

이 노트북 내용은 인라이플 버트이해하기 발표자료, [BERT 톺아보기 블로그](http://docs.likejazz.com/bert/), [텐서플로와 머신러닝으로 시작하는 자연어 처리 (전창욱, 최태균, 조중현)](https://book.naver.com/bookdb/book_detail.nhn?bid=14488487)를 참고 했음을 밝힙니다.

BERT(Bidirectional Encoder Representations from Transformers)는 2018년 10월에 논문이 공개됐고, 11월에 오픈소스로 코드가 공개된 구글의 새로운 Language Representation Model입니다. 

<img src="https://user-images.githubusercontent.com/31824102/36531999-78eda612-17b7-11e8-9793-ef8dcd78dd9b.PNG"></img>

BERT는 Transformer 기반입니다. 인코더-디코더 모델이며 복잡합니다. BERT는 인코더만 사용합니다. 

## Positional Encoding (Positional Embedding)

Transformer는입력으로 토큰의 상대적 또는 절대적 위치에 대한 정보를 주입해야 합니다. Positional Encoding이라고 합니다. 사인, 코사인 함수를 이용하여 토큰의 상대적 위치와 관련한 정보를 학습할 수 있게 됩니다.

\begin{align}
PE(pos,2i) &= sin( pos/10000^{2i/dmodel}) \\
PE(pos,2i+1) &= cos( pos/10000^{2i/dmodel} )
\end{align}

BERT는 Transformer와 달리 Positional Encoding을 사용하지 않고 Position Embedding을 사용합니다. 입력 데이터는 두 문장 합쳐 약 512단어 이하로 넣고 3가지 형식으로 입력하여 임베딩합니다.

*   토큰 임베딩: 입력 단어가 단어장(Vocab)에 몇 번째 단어인지 나타냄
*   세그먼트 임베딩: 두 문장 중 앞 문장인지 뒷 문장인지 구분함
*   포지션 임베딩: 문장 내 단어의 순번을 나타냄

특수 기호 토큰들
*  0 PAD: 공백
*  1 CLS: 첫 토큰으로 항상 CLS를 넣음
*  2 UNK: 단어장에 없는 단어
*  3 SEP: 문장들을 구분해 주는 구분자
*  4 MASK: 마스크 처리

3개의 임배딩을 합산한 결과를 취합니다.

```
e = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
```

```pos```는 각 토큰의 위치 정보이며 range(0, max_len)이며 ```seg```는 토큰 타입입니다. 이제 이 값에 대한 각각의 임베딩을 얻어 합산하고 여기에 LayerNorm & Dropout 한 최종 결과를 인코더 블럭에 입력합니다.

## Encoder block

<img src="https://cdn-images-1.medium.com/max/1600/1*EblTBhM-9mOqYWMARk6ajQ.png" />

인코더 블록에서는 임베딩 결과를 가지고 셀프 어텐션과 피드 포워드 작업을 수행한다. 이 블록은 모델마다 개수가 다른데 Bert의 base 모델은 12개를 사용한다. 즉, 입력 시퀀스를 12번 반복한다는 의미이다. 

### Multi-Head Attention

인코더에서 제일 중요하다. Multi-Head Attention은 여러 해드를 가진 어텐션을 의미한다. 우선 어텐션의 개념을 배워야 한다. 

```
Attention is all you need?
어텐션은 딥러닝 모델에서 특정 벡터에 주목하도록 하여 모델의 성능을 높이는 기법이다.  
```

서로 다른 가중치 행렬(헤드)를 $h$(헤드 개수)만큼 어텐션 계산을 한 다음 서로 연결하여 결과를 만든다. Bert base모델에서는 12개 헤드를 사용한다. 768개 토큰을 12등분하여 64개 토큰을 12개 헤드에 각각 계산하고 Scaled-Dot Product를 사용하여 768로 다시 합친다.  